SUBMITTED BY-JOYAL PASRICHA
ROLL NUMBER: DA24C008
FUNDAMENTALS OF MACHINE LEARNING ASSIGNMENT 2: SPAM HAM CLASSIFIER

Importing required libraries

In [2]:
import regex as re
import string
import numpy as np
import pandas as pd
import os

importing the training dataset

In [ ]:
#importing data from the csv file called "emails.csv"
df = pd.read_csv("C:\\Users\\Joyal Pasricha\\Downloads\\emails.csv", header=None, names=['text', 'label'])
#converting labels to numeric values 0 and 1 where 0 is for ham or non spam and 1 for spam
df['label'] = df['label'].map({'ham': 0,'spam': 1})

Preprocessing step-
We preprocess the text by removing the stop  by making a function called "remove_stop_words"

In [4]:
def remove_stop_words(text):
    stop_words = set(['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 
                      'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 
                      'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 
                      'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 
                      'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 
                      'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 
                      'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 
                      'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 
                      'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 
                      'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 
                      'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 
                      'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 
                      'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 
                      'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', 
                      "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', 
                      "wouldn't"])
    
    words = str(text).lower().split()
    filtered_words = [word.strip(string.punctuation) for word in words if word not in stop_words and not re.fullmatch(r'[^\w\s]+', word)]
    return ' '.join(filtered_words)

Class for textvectorizer

In [ ]:
class TextVectorizer:
    def __init__(self):
        self.vocab = {}

    def fit(self, corpus):
        vocab = set(word.lower() for sentence in corpus for word in sentence.split())
        self.vocab = {word: i for i, word in enumerate(vocab)}

    def transform(self, corpus):
        ret_val = np.zeros((len(corpus), len(self.vocab)))
        for i, sentence in enumerate(corpus):
            for word in sentence.split():
                if word in self.vocab:
                    ret_val[i, self.vocab[word]] += 1  #count heere is used for word frequency
        return ret_val

    def fit_transform(self, corpus):
        self.fit(corpus)
        return self.transform(corpus)



Class for Naive bayes classifier

In [6]:

class NaiveBayesClassifier:
    def __init__(self):
        self.word_prob = {0: {}, 1: {}}
        self.total_spam = 0
        self.total_ham = 0

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.total_spam = np.sum(y == 1)
        self.total_ham = np.sum(y == 0)

        # Prevent division by zero
        self.word_prob[1] = (np.sum(X[y == 1], axis=0) + 1) / (self.total_spam + n_features) if self.total_spam > 0 else np.zeros(n_features)
        self.word_prob[0] = (np.sum(X[y == 0], axis=0) + 1) / (self.total_ham + n_features) if self.total_ham > 0 else np.zeros(n_features)

    def predict(self, X_test):
        log_spam = np.log(self.total_spam / (self.total_spam + self.total_ham)) if self.total_spam > 0 else float('-inf')
        log_ham = np.log(self.total_ham / (self.total_spam + self.total_ham)) if self.total_ham > 0 else float('-inf')

        y_predict = []
        for x in X_test:
            log_prob_spam = np.sum(x * np.log(self.word_prob[1])) if self.total_spam > 0 else float('-inf')
            log_prob_ham = np.sum(x * np.log(self.word_prob[0])) if self.total_ham > 0 else float('-inf')

            if log_prob_spam + log_spam > log_prob_ham + log_ham:
                y_predict.append(1)  # Spam
            else:
                y_predict.append(0)  # Ham
        return np.array(y_predict)

now applying these functions on the training dataset

In [10]:

#Preprocessing the training text
df["text"] = df["text"].apply(remove_stop_words)

#Vectorization for training data using the funtcions made previously
vectorizer = TextVectorizer()
X_train = vectorizer.fit_transform(df["text"])
y_train = df["label"].values

#Training the classifier using naive bayes algorithm.
naivebayes = NaiveBayesClassifier()
naivebayes.fit(X_train, y_train)

Function that would read emails from the 'test' folder and predicts the labels

In [ ]:
def classify_emails(folder_path):
    predictions = {}
    for filename in os.listdir(folder_path):
        if filename.endswith('.txt'):
            with open(os.path.join(folder_path, filename), 'r', encoding='utf-8') as file:
                email_content = file.read()
                cleaned_email = remove_stop_words(email_content)
                X_test = vectorizer.transform([cleaned_email])
                prediction = naivebayes.predict(X_test)
                predictions[filename] = 1 if prediction[0] == 1 else 0
    return predictions



Finally, printing the email names and their labels (1 for spam,0 for non spam/ham)

In [9]:
# Specify the test folder and classify emails
test_folder = 'test'
results = classify_emails(test_folder)

# Print results
for email, pred in results.items():
    print(f"{email}: {'Spam' if pred == 1 else 'Non-Spam'}")

email1.txt: Spam
